In [1]:
import requests

from config import SPOT_CLIENT_SECRET, SPOT_CLIENT_ID
CLIENT_ID = SPOT_CLIENT_ID
CLIENT_SECRET = SPOT_CLIENT_SECRET


def get_spotify_token(client_id, client_secret):
    auth_response = requests.post(
        'https://accounts.spotify.com/api/token',
        {'grant_type': 'client_credentials'},
        auth=(client_id, client_secret)
    )
    return auth_response.json()['access_token']

def search_track(track_name, artist_name, token):
    headers = {'Authorization': f'Bearer {token}'}
    search_query = f'track:{track_name} artist:{artist_name}' if artist_name else f'track:{track_name}'
    search_params = {
        'q': search_query,
        'type': 'track',
        'limit': 1
    }
    response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=search_params)
    return response.json()

# credenciais do Spotify
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

# Obter o token
token = get_spotify_token(client_id, client_secret)

# Buscar uma música 
track_name = 'Livin On A Prayer'
artist_name = 'Bon Jovi'
track_info = search_track(track_name, artist_name, token)


In [2]:
def get_track_features(track_id, token):
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(f'https://api.spotify.com/v1/audio-features/{track_id}', headers=headers)
    return response.json()

#ID da música do passo anterior
track_id = track_info['tracks']['items'][0]['id']
features = get_track_features(track_id, token)


In [3]:
print(features)

#detalhes da faixa a partir da API
track_details = search_track(track_name, artist_name, token)

# nome e o artista corretamente
track_info = {
    'name': track_details['tracks']['items'][0]['name'],
    'artist': track_details['tracks']['items'][0]['artists'][0]['name'],
    'album': track_details['tracks']['items'][0]['album']['name'],  # Nome do Álbum
    'album_release_date': track_details['tracks']['items'][0]['album']['release_date'],  # Data de Lançamento do Álbum
    'album_type': track_details['tracks']['items'][0]['album']['album_type']  # Tipo do Álbum (ex: single, album)
    # outras informações conforme necessário
}


{'danceability': 0.532, 'energy': 0.887, 'key': 0, 'loudness': -3.757, 'mode': 1, 'speechiness': 0.0335, 'acousticness': 0.0778, 'instrumentalness': 0.000214, 'liveness': 0.294, 'valence': 0.795, 'tempo': 122.511, 'type': 'audio_features', 'id': '37ZJ0p5Jm13JPevGcx4SkF', 'uri': 'spotify:track:37ZJ0p5Jm13JPevGcx4SkF', 'track_href': 'https://api.spotify.com/v1/tracks/37ZJ0p5Jm13JPevGcx4SkF', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/37ZJ0p5Jm13JPevGcx4SkF', 'duration_ms': 249293, 'time_signature': 4}


In [4]:
# modelo carregado
import pandas as pd
import joblib

# Carregar o scaler e o label_encoder salvos
scaler = joblib.load('preprocessor.joblib')
label_encoder = joblib.load('label_encoder.joblib')

# Carregar os novos dados

new_data = pd.DataFrame([features])
relevant_columns = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'mode', 'time_signature']
dados_novos = new_data[relevant_columns]


# Aplicação do Pre-processamento

X_new_processed = scaler.transform(dados_novos)

# Carga do modelo

best_knn = joblib.load('knn_best_model.joblib')
best_svm = joblib.load('svm_best_model.joblib')
best_nb = joblib.load('gaussian_nb_best_model.joblib')
best_rf = joblib.load('random_forest_best_model.joblib')
best_gb = joblib.load('gradient_boosting_best_model.joblib')
best_xgb = joblib.load('xgboost_best_model.joblib')

# Fazer previsões com o modelo
quadrant_prediction_knn = best_knn.predict(X_new_processed)
quadrant_prediction_svm = best_svm.predict(X_new_processed)
quadrant_prediction_nb = best_nb.predict(X_new_processed)
quadrant_prediction_rf = best_rf.predict(X_new_processed)
quadrant_prediction_gb = best_gb.predict(X_new_processed)
quadrant_prediction_xgb = best_xgb.predict(X_new_processed)

# Se necessário, transformar as previsões de volta para os rótulos originais
#y_new_pred_labels = label_encoder.inverse_transform(y_new_pred)

#from tensorflow.keras.models import load_model

##neural_model_loaded = load_model('nn_model.keras')
#model_loaded = joblib.load('melhor_knn_model.joblib')

# Preparar os dados

#import pandas as pd
#from sklearn.preprocessing import StandardScaler

# 'features' é um dicionário com as características extraídas da API do Spotify
#new_data = pd.DataFrame([features])

# Selecionar apenas as colunas relevantes
#relevant_columns = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
#new_data = new_data[relevant_columns]

# Aplicando a normalização
#scaler = StandardScaler()
#new_data[relevant_columns] = scaler.fit_transform(new_data[relevant_columns])

# Aplicar a mesma normalização/padronização usada no treinamento
#new_data_scaled = scaler.transform(new_data)

# Fazer a previsão
#quadrant_prediction = model_loaded.predict(new_data_scaled)


In [5]:


quadrants = {
    0: 'Q1 - Alta Valência, Alta Ativação (Alegre, Energética)',
    1: 'Q2 - Baixa Valência, Alta Ativação (Tensa, Nervosa)',
    2: 'Q3 - Alta Valência, Baixa Ativação (Triste, Melancólica)',
    3: 'Q4 - Baixa Valência, Baixa Ativação (Calma, Relaxante)',
 'Q1': 'Q1 - Alta Valência, Alta Ativação (Alegre, Energética)',
 'Q2': 'Q2 - Baixa Valência, Alta Ativação (Tensa, Nervosa)',
 'Q3': 'Q3 - Alta Valência, Baixa Ativação (Triste, Melancólica)',
 'Q4': 'Q4 - Baixa Valência, Baixa Ativação (Calma, Relaxante)'
}

# 'quadrants' é uma lista ou dicionário que mapeia os números da previsão para os rótulos dos quadrantes
predicted_quadrant_knn = quadrants[quadrant_prediction_knn[0]]
predicted_quadrant_svm = quadrants[quadrant_prediction_svm[0]]
predicted_quadrant_nb = quadrants[quadrant_prediction_nb[0]]
predicted_quadrant_rf = quadrants[quadrant_prediction_rf[0]]
predicted_quadrant_gb = quadrants[quadrant_prediction_gb[0]]
predicted_quadrant_xgb = quadrants[quadrant_prediction_xgb[0]]

# Combinando as informações
track_info['predicted_quadrant_knn'] = predicted_quadrant_knn
track_info['predicted_quadrant_svm'] = predicted_quadrant_svm
track_info['predicted_quadrant_nb '] = predicted_quadrant_nb 
track_info['predicted_quadrant_rf '] = predicted_quadrant_rf 
track_info['predicted_quadrant_gb '] = predicted_quadrant_gb 
track_info['predicted_quadrant_xgb'] = predicted_quadrant_xgb

# Exibindo as informações
print(f"Nome da Música: {track_info['name']}")
print(f"Artista: {track_info['artist']}")
print(f"Album:{track_info['album']}")
print(f"\nQuadrante Previsto_knn: {track_info['predicted_quadrant_knn']}")
print(f"Quadrante Previsto_svm: {track_info['predicted_quadrant_svm']}")
print(f"Quadrante Previsto_nb : {track_info['predicted_quadrant_nb ']}")
print(f"Quadrante Previsto_rf : {track_info['predicted_quadrant_rf ']}")
print(f"Quadrante Previsto_gb : {track_info['predicted_quadrant_gb ']}")
print(f"Quadrante Previsto_xgb: {track_info['predicted_quadrant_xgb']}")

# ... exibir outras informações desejadas


Nome da Música: Livin' On A Prayer
Artista: Bon Jovi
Album:Slippery When Wet

Quadrante Previsto_knn: Q2 - Baixa Valência, Alta Ativação (Tensa, Nervosa)
Quadrante Previsto_svm: Q1 - Alta Valência, Alta Ativação (Alegre, Energética)
Quadrante Previsto_nb : Q1 - Alta Valência, Alta Ativação (Alegre, Energética)
Quadrante Previsto_rf : Q1 - Alta Valência, Alta Ativação (Alegre, Energética)
Quadrante Previsto_gb : Q2 - Baixa Valência, Alta Ativação (Tensa, Nervosa)
Quadrante Previsto_xgb: Q2 - Baixa Valência, Alta Ativação (Tensa, Nervosa)


In [8]:
import numpy as np

# Dicionário de mapeamento dos números da previsão para os rótulos dos quadrantes
quadrants = {
    0: 'Alta Valência, Alta Ativação (Alegre, Energética)',
    1: 'Baixa Valência, Alta Ativação (Tensa, Nervosa)',
    2: 'Baixa Valência, Baixa Ativação (Triste, Melancólica)',
    3: 'Alta Valência, Baixa Ativação (Calma, Relaxante)'
}

# Supondo que `quadrant_prediction` é o resultado da rede neural
# Aqui você deve ter o resultado da chamada `model.predict(...)`
# quadrant_prediction = model.predict(some_input_data)

# Obtenha o índice da classe com a maior probabilidade para a previsão
predicted_class_index = np.argmax(quadrant_prediction, axis=-1)

# Agora você pode acessar o dicionário usando esse índice
# Note que `predicted_class_index` é um array, então você acessa o primeiro elemento com [0]
predicted_quadrant = quadrants[predicted_class_index[0]]

# Supondo que 'track_info' é um dicionário contendo informações da música
# track_info = {'name': 'Name of the Song', 'artist': 'Name of the Artist', ...}

# Adicione o quadrante previsto ao dicionário de informações da música
track_info['predicted_quadrant'] = predicted_quadrant

# Exiba as informações
print(f"Nome da Música: {track_info['name']}")
print(f"Artista: {track_info['artist']}")
print(f"Quadrante Previsto: {track_info['predicted_quadrant']}")
# ... exibir outras informações desejadas


IndexError: invalid index to scalar variable.